## Used to simulate the entire flow (Hybrid HHL and Time Locked Puzzle)


![Workflow Diagram](./diagram_of_scheme.png)


In [43]:
from enhanced_hybrid_hhl import (
    QuantumLinearSystemProblem,
    run_braket_projection_submission,
    run_braket_projection_retrieval
)
import os
import numpy as np
import json
import time
from puzzle_generation.tlp_lattice import generate_puzzle, solve_puzzle

## Initialize

In [54]:
A_matrix = np.array([
    [2, 0, 0, 4],
    [0, 2, 0, 0],
    [0, 0, 2, 0],
    [4, 0, 0, 2]
])

b_vector = np.array([1.0, 2.0, 0.0, 1.5])

# A_matrix = np.array([
#     [2, 0],
#     [0, 2]
# ])

# b_vector = np.array([1.0, 1.0])



T = 4500 *20 # 4500 == 1s of execution
message = b"This is a secret msg for HHL!"

print( "Conditional number (kappa) of A:", np.linalg.cond(A_matrix) )
print( "Eigenvalues of A:", np.linalg.eigvals(A_matrix) )

print("[*] Generating puzzle...")
puzzle = generate_puzzle(T, message)
with open("./puzzle_generation/puzzle.json", "w") as f:
    json.dump(puzzle, f)
print("Generated !")

Conditional number (kappa) of A: 3.000000000000001
Eigenvalues of A: [ 6. -2.  2.  2.]
[*] Generating puzzle...
Generated !


In [49]:
problem1 = QuantumLinearSystemProblem(A_matrix, b_vector)

# Setup output directory

output_dir = os.path.join("./", "examples/hardware_results/aws_braket")

# Submit
print("Submitting projection measurements to AWS Braket...")
result_path = run_braket_projection_submission(
    problem_list=[problem1],
    output_dir=output_dir,
    k_qubits=3,
    shots=200,
    preprocessing_mode="ideal"
)


Submitting projection measurements to AWS Braket...
✓ AWS Braket Backend: SV1
  Region: eu-west-2
  Problems: 1
  Solution size: 4 (2 qubits)
  Projection mode: Measuring all 4 components

--- Processing 1 problems ---

[1/1] Problem 1
  Using ideal preprocessing (classical)
  → Submitting 4 projection measurements...
    [1/4] Basis |0⟩: arn:aws:braket:eu-west-2:380012739362:quantum-task/51b760c1-b14c-4791-b092-091a1133360e
    [2/4] Basis |1⟩: arn:aws:braket:eu-west-2:380012739362:quantum-task/9b5316ab-a762-4981-8650-19974245e2f8
    [3/4] Basis |2⟩: arn:aws:braket:eu-west-2:380012739362:quantum-task/496b8e30-2c39-490e-9ae3-5093438c1fb4
    [4/4] Basis |3⟩: arn:aws:braket:eu-west-2:380012739362:quantum-task/f302f7be-8a82-4d87-b26d-bce847a692e4
  Circuit depth: 251
  Submitted 4 measurements

✓ Results saved to: braket_projection_N4_hhl_20251218_172226.json
  Total tasks submitted: 4


In [53]:
print("\nRetrieving results from AWS Braket...")
retrieval_path = run_braket_projection_retrieval(
    result_file_path="./examples/hardware_results/aws_braket/braket_projection_N16_hhl_16x16Task.json"
)


Retrieving results from AWS Braket...
Loading projection results from: braket_projection_N16_hhl_16x16Task.json
✓ Using AWS Braket Backend: SV1
  Region: eu-west-2

--- Retrieving Results from AWS Braket ---

Problem 1/1
  Retrieving 16 projection measurements...

  Debug: Target |0⟩ = |0000⟩
  Bitstring: 0100000000001, flag=1, solution=0100
  Success rate: 0.2800
  P(target|success): 0.1429
    Basis |0⟩: P=0.1429, Success=0.2800, Time=9.1s
    Basis |1⟩: P=0.0297, Success=0.3367, Time=8.8s
    Basis |2⟩: P=0.1143, Success=0.3500, Time=11.0s
    Basis |3⟩: P=0.0112, Success=0.2967, Time=11.2s
    Basis |4⟩: P=0.2000, Success=0.3500, Time=10.6s
    Basis |5⟩: P=0.0093, Success=0.3600, Time=11.2s
    Basis |6⟩: P=0.0000, Success=0.3000, Time=9.2s
    Basis |7⟩: P=0.0278, Success=0.3600, Time=8.8s
    Basis |8⟩: P=0.1100, Success=0.3333, Time=8.6s
    Basis |9⟩: P=0.0220, Success=0.3033, Time=10.5s
    Basis |10⟩: P=0.0000, Success=0.3567, Time=8.6s
    Basis |11⟩: P=0.0092, Success=0.3

In [19]:
print("[*] Solving puzzle...")
with open("./puzzle_generation/puzzle.json", "r") as f:
    loaded_puzzle = json.load(f)


start_time = time.time()
recovered = solve_puzzle(loaded_puzzle)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Solved in {elapsed_time:.2f} seconds")
print("    Recovered Message:", recovered.decode())

[*] Solving puzzle...
Solved in 2.08 seconds
    Recovered Message: This is a secret msg for HHL!
